# Import Libs

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os,warnings,csv,shutil
import cv2
import math

from tensorflow.keras.models import model_from_json

# Load the trained models

In [ ]:

# load json and create model
json_file = open('/kaggle/input/validate/MultiScale-Triplet.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
multiscale = model_from_json(loaded_model_json)
# load weights into new model
multiscale.load_weights("../input/validate/35e.h5")
print("Loaded model from disk")


# load json and create model
json_file = open('/kaggle/input/validate/Classifier.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
classifier = model_from_json(loaded_model_json)
# load weights into new model
classifier.load_weights("../input/validate/class35e.h5")
print("Loaded model from disk")

# Map <labels,specie code>

In [ ]:
labels = np.load("/kaggle/input/birdclef-2020/labels.npy")
filenames = np.load("/kaggle/input/birdclef-2020/filenames.npy")

mapa = {}
top = np.unique(labels)

cont = 0
for i in top:
    mapa[i] = cont
    cont +=1        
    
inv_mapa = {v: k for k, v in mapa.items()}
#mapa
inv_mapa

In [ ]:
name_to_cod = {}
with open('/kaggle/input/birdclef-2020/Validation/Validation/species_names_to_codes.txt', 'r') as file:
    for row in file:
        name,cod = row.split(',')
        cod = cod.split('\n')
        name_to_cod[name] = cod[0]  

# Get species per place

In [ ]:
ger = {""}

with open('/kaggle/input/validate/species_list_GER.txt', 'r') as file:
    for row in file:
        row = row.split('\n')
        ger.add(name_to_cod[row[0]])
        
hsn = {""}

with open('/kaggle/input/validate/species_list_HSN.txt', 'r') as file:
    for row in file:
        row = row.split('\n')
        hsn.add(name_to_cod[row[0]])
        
per = {""}

with open('/kaggle/input/validate/species_list_PER.txt', 'r') as file:
    for row in file:
        row = row.split('\n')
        per.add(name_to_cod[row[0]])
        
ssw = {""}

with open('/kaggle/input/validate/species_list_SSW.txt', 'r') as file:
    for row in file:
        row = row.split('\n')
        ssw.add(name_to_cod[row[0]])        

species_loc = {'GER':ger, 'HSN':hsn, 'PER':per, 'SSW':ssw}        

# Predicting

In [ ]:
def my_predict(y_in_perc,csv_file,audio):

    dez = y_in_perc.argsort()[-30:][::-1]
    
    m = 0.0
    cont = 0
    for i in range(len(dez)):
        if(inv_mapa[dez[i]] in species_loc[audio[:3]]):
            writer.writerow([audio[:5],audio[5:22],inv_mapa[dez[i]],(1.0)])
            cont += 1
            if(cont == 10):
                break
            m += 0.1
    

In [ ]:
import csv

with open('sub.csv', 'w', newline='') as file:
    writer = csv.writer(file,delimiter=';')

    path_audios = '/kaggle/input/testea/test_chunks/'
    audios = [rec for rec in os.listdir(path_audios)]
    audios.sort()

    for audio in audios:
        dict_data = np.load(path_audios+audio)
        data = dict_data['arr_0']
        val_input = multiscale.predict([[data]])
        val_input = val_input[0]

        pred = classifier.predict([[val_input]])
        
        audio = audio.replace('a',':')
        my_predict(pred[0],writer,audio)